In [1]:
import pandas as pd
from random import sample
from tqdm.auto import tqdm

tqdm.pandas()

df_review = pd.read_csv(r"/home/karysoares/Documents/book-reviews/data/books_rating.csv")
df_books_data = pd.read_csv(r"/home/karysoares/Documents/book-reviews/data/books_data.csv")

In [2]:
# Se tentarmos comparar os "Title" diretamente, estaremos comparando strings - uma operação custosa.
# Então, vamos converter o "title" para hashes. Comparar hashes é uma operação muito menos custosa do que 
# comparar strings. Isso poderia ser feito de outro modo, como usando 'merge', mas desse modo as operações
# ficam mais transparentes.
df_review['title_hash'] = df_review['Title'].apply(lambda x: hash(x))

In [3]:
def find_reviews(title):
    title_hash = hash(title)
    return df_review[df_review['title_hash'] == title_hash]

In [4]:
df_books_data['review'] = df_books_data['Title'].progress_apply(find_reviews)

  0%|          | 0/212404 [00:00<?, ?it/s]

In [ ]:
# vamos já preparar a média dos reviews para adiantar essa operação nos outros notebooks
df_books_data['review_mean'] = df_books_data['review'].apply(lambda x: x['score'].mean())

In [ ]:
# Setando o Index do df_review para facilitar a indexação em uma db relacional
df_review.set_index("Id", drop=True, inplace=True)

In [ ]:
from sqlalchemy import create_engine

# Vamos salvar os dados para garantir que as operações não sejam em vão
# Como simplesmente adicionamos um recorte do df_review diretamente no df_books_data, 
# precisaremos simplificar a relação entre os datasets. Para isso, substituiremos 
# o dataframe por uma lista de ids 

## Aqui nós iremos guardar os ids como uma string representando uma lista. 
## Sqlite suporta json_fields, mas guardá-lo utilizando pandas seria bem chato. 
## Como carregaremos novamente com o pandas, só precisaremos converter os valores do outro lado.
df_books_data['review_ids'] = df_books_data['review'].apply(lambda x: str(list(x['Id'])))
# Não queremos salvar todo o dataframe, então vamos removê-lo do df_books_data
del df_books_data['review']


engine = create_engine("sqlite:///../books.db")
df_review.to_sql("books_review", engine, if_exists="replace", index=True) # queremos que o index do dataframe seja utilizado aqui.
df_books_data.to_sql("books_data", engine, if_exists="replace", index=False)

In [ ]:
df_books_data = df_books_data[df_books_data['description'].notnull()]
df_books_data = df_books_data[df_books_data['categories'].notnull()]

In [ ]:
# Como todos os resultados só tem uma categoria, vamos colapsá-la em uma string (ao invés de uma lista)
df_books_data['categories'] = df_books_data['categories'].apply(eval)
print(df_books_data['categories'].apply(len).max())
df_books_data['categories'] = df_books_data['categories'].apply(lambda x: x[0])

In [ ]:
df_category = pd.DataFrame(
    {
        "text": df_books_data['description'],
        "label": df_books_data['categories'].str.lower(),
    }
)

In [ ]:
# Muitas labels aparecem poucas vezes e devemos considerá-las como "dados ruins". Iremos eliminar todas as linhas que tiverem labels que aparecem menos de 100 vezes.
label_count = df_category['label'].value_counts()
label_count = label_count[label_count >= 100]
df_category = df_category[df_category['label'].isin(label_count.keys())]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(df_category.sample(frac=0.2), test_size=0.2)

In [ ]:
train.to_csv('../data/category_train.csv', index=False)
test.to_csv('../data/category_test.csv', index=False)

In [ ]:
df_review.head()

In [ ]:
df_sentiment = pd.DataFrame({
    "text": df_review['text'],
    "label": df_review['score'],
})

In [ ]:
print("Total de linhas: ", df_sentiment.size)
print("Total de linhas com text", df_sentiment[df_sentiment['text'].notnull()].size)
print("Total de linhas com score", df_sentiment[df_sentiment['label'].notnull()].size)

In [ ]:
df_sentiment = df_sentiment[df_sentiment['text'].notnull()]

In [ ]:
df_sentiment.label = df_sentiment.label.apply(int)

In [ ]:
train, test = train_test_split(df_sentiment.sample(frac=0.5), test_size=0.2)

In [ ]:
train.to_csv('../data/sentiment_train.csv', index=False)
test.to_csv('../data/sentiment_test.csv', index=False)

In [ ]:
df_sentiment['label'].unique()